In [1]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [2]:
from pymongo import MongoClient
from bson.objectid import ObjectId

import time,datetime
import os

import pandas as pd
import pandas_datareader.data as web
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import style

from keras.models import Model, Sequential
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

from sklearn import preprocessing

from copy import deepcopy
from IPython.display import clear_output

In [3]:
style.use('ggplot')

In [4]:
conn = MongoClient()
db = conn.Tracker
collection = [db.james]# db.db2, db.dn2, db.james, db.leo
clean_data = []
for col in collection:
    cursor = col.find({})
    df = pd.DataFrame(list(cursor))
df.replace('', np.nan, inplace=True)
df.fillna(method='ffill', inplace=True)

df['hr_value'] = df['hr_value'].astype(float)
df['o2_value'] = df['o2_value'].astype(float)
df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)
df['step_value'] = df['step_value'].astype(float)

# map out the weekday
df['weekday'] = df['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x).strftime('%w'))
df['week'] = df['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x).isocalendar()[1]
          - (datetime.datetime.fromtimestamp(x).isoweekday() < 7)) # do this so that week may start on monday

In [5]:
dfsteptotal = pd.DataFrame({})
dfsteptotal = df[['week', 'weekday']].set_index(['week', 'weekday'])
dfsteptotal = dfsteptotal[~dfsteptotal.index.duplicated(keep='first')]
dfsteptotal.reset_index(inplace=True)
# this is where the data have null
dfadd = pd.DataFrame({'week' : [12], 'weekday':['5']})
dfsteptotal = dfsteptotal.append(dfadd, ignore_index=True)
dfsteptotal = dfsteptotal.set_index(['week', 'weekday'])
dfsteptotal.sort_index(inplace=True)
dfsteptotal['msteps'] = 0
dfsteptotal['asteps'] = 0
dfsteptotal['esteps'] = 0

In [6]:
# here I map between integer and the sting it's mean
timeslidemap = {0:'msteps', 1:'asteps', 2:'esteps'}
basestamp = df['timestamp'][0]
endstamp = df['timestamp'][-1:].values[0]
# for num, row in df[:30000].iterrows():
while True:
    try:
        estimedevide = int(int(datetime.datetime.fromtimestamp(basestamp).strftime('%H')) / 8)
        hourstart = 8*estimedevide
        hourend = 8*estimedevide + 7
        dfbetween = df.loc[df['timestamp'].between(
            datetime.datetime.fromtimestamp(basestamp).replace(hour=hourstart,minute=0,second=0).timestamp(),
            datetime.datetime.fromtimestamp(basestamp).replace(hour=hourend,minute=59,second=59).timestamp()
                    , inclusive=True)]
        steps = dfbetween['step_value'].sum()
    #     print(int(datetime.datetime.fromtimestamp(basestamp).strftime('%m')),
    #             int(datetime.datetime.fromtimestamp(basestamp).strftime('%d')),
    #                 timeslidemap[estimedevide])
    
        dfsteptotal.loc[[(datetime.datetime.fromtimestamp(basestamp).isocalendar()[1] - 
                                    (datetime.datetime.fromtimestamp(basestamp).isoweekday() < 7),
                            datetime.datetime.fromtimestamp(basestamp).strftime('%w'))], 
                                           [timeslidemap[estimedevide]]] += steps

        basestamp = (datetime.datetime.fromtimestamp(basestamp) + datetime.timedelta(hours=8)).timestamp()
        if basestamp > endstamp:
            break
    except Exception as e:
        print(str(e))
        basestamp = (datetime.datetime.fromtimestamp(basestamp) + datetime.timedelta(hours=8)).timestamp()
#     else:
#         print(datetime.datetime.fromtimestamp(basestamp) ,datetime.datetime.fromtimestamp(endstamp))

In [7]:
dfsteptotal

msteps  asteps  esteps
week weekday                        
12   3           0.0     0.0  2366.0
     4           0.0  2104.0     0.0
     5           0.0     0.0     0.0
     6           0.0  1878.0  1215.0
13   0         320.0   454.0   367.0
     1           0.0  8157.0  2275.0
     2         248.0  3835.0  2857.0
     3           0.0  2073.0  3846.0
     4         499.0  3090.0  1890.0
     5         318.0  2402.0  1471.0
     6           0.0  2168.0  2975.0
14   0         692.0  2506.0  5639.0
     1         289.0  3837.0  1581.0
     2           0.0   229.0  3903.0
     3           0.0  4238.0  1673.0
     4           0.0  1913.0   934.0
     5          91.0  3629.0  1915.0
     6        1391.0  3726.0  1000.0
15   0         229.0  4541.0  3494.0
     1        1111.0  8982.0  3654.0
     2           0.0  2018.0  3909.0
     3         298.0  4418.0  2597.0
     4         342.0  5346.0  3889.0
     5         327.0  4217.0  1638.0
     6          53.0  5907.0  1855.0
16   0         300.0  3643.0  1872.0
     1         503.0  9657.0  1355.0
     2           0.0  2395.0  4981.0
     3          28.0  5646.0  2604.0
     4           0.0  2459.0  1616.0